In [26]:
# ---- my_image_processing.ipynb ----

import os
import random
from PIL import Image

# ========== 配置部分 ==========
IMG_FOLDER = "/Users/lixiaoyu/Downloads/L历代山水4/input"       # 待处理图片所在文件夹
OUTPUT_FOLDER = "/Users/lixiaoyu/Downloads/L历代山水4/output"  # 样本保存的目标文件夹
SHORT_SIDE_TARGET = 1024          # 短边希望达到的大小
CROP_SIZE = 512                   # 每个样本的宽高
NUM_CROPS = 5                     # 每张图在中心附近截几张
RANDOM_SEED = 42                  # 固定随机种子，保证可复现

# 让随机结果可复现（如果你想每次都不一样，可以注释掉这行）
random.seed(RANDOM_SEED)

def process_image(image_path, output_folder,
                  short_side_target=1024,
                  crop_size=512,
                  num_crops=5):
    """
    1) 如果图的短边 < short_side_target，等比放大到短边=short_side_target
    2) 然后在图像中心附近随机截取 num_crops 张 512x512 样本
    """
    img = Image.open(image_path)
    w, h = img.size

    # ============ 1) 按比例缩放 ============
    short_side = min(w, h)
    if short_side < short_side_target:
        pass
    else:
        scale_ratio = short_side_target / float(short_side)
        new_w = int(w * scale_ratio)
        new_h = int(h * scale_ratio)
        img = img.resize((new_w, new_h), Image.LANCZOS)
        w, h = new_w, new_h
        pass

    # 如果此时图片还是小于 crop_size，就无法裁剪，直接跳过
    if w < crop_size or h < crop_size:
        print(f"[跳过] {image_path}：图片尺寸 {w}x{h} 不足 {crop_size}。")
        return

    # ============ 2) 从中心附近随机截取多个patch ============
    # 图像中心坐标
    center_x = w // 2
    center_y = h // 2

    # 随机偏移的最大范围，确保不会超出图像边界
    max_offset_x = max((w - crop_size) // 2, 0)
    max_offset_y = max((h - crop_size) // 2, 0)

    # 第1张：严格中心裁剪
    left = center_x - crop_size // 2
    top  = center_y - crop_size // 2
    patch_center = img.crop((left, top, left + crop_size, top + crop_size))

    # 输出文件名
    basename = os.path.splitext(os.path.basename(image_path))[0]
    patch_center.save(os.path.join(output_folder, f"{basename}_center.png"))

    # 如果只要 1 张，就结束函数
    if num_crops == 1:
        return

    # 其余若干张：在中心附近随机偏移
    for i in range(num_crops - 1):
        offset_x = random.randint(-max_offset_x, max_offset_x)
        offset_y = random.randint(-max_offset_y, max_offset_y)

        left = center_x - (crop_size // 2) + offset_x
        top  = center_y - (crop_size // 2) + offset_y

        # 防止越界
        left = max(0, min(left, w - crop_size))
        top  = max(0, min(top, h - crop_size))

        patch = img.crop((left, top, left + crop_size, top + crop_size))

        out_name = os.path.join(output_folder, f"{basename}_rand{i}.png")
        patch.save(out_name)

def batch_process_images(img_folder=IMG_FOLDER, output_folder=OUTPUT_FOLDER):
    """
    批量处理文件夹内所有图片，依次调用 process_image
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    valid_exts = (".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff", ".webp")

    # 遍历文件夹所有图片
    for filename in os.listdir(img_folder):
        if filename.lower().endswith(valid_exts):
            image_path = os.path.join(img_folder, filename)
            process_image(image_path,
                          output_folder,
                          short_side_target=SHORT_SIDE_TARGET,
                          crop_size=CROP_SIZE,
                          num_crops=NUM_CROPS)
            print(f"处理完：{filename}")

# ========== 在 notebook 里直接调用 ==========
# 如果你已经把图片放到 "input_images" 文件夹中，可以执行:
batch_process_images()
print("全部处理完成。")


处理完：元  赵孟頫 蜀道难 绢本 56.6x154.45.jpg
处理完：五代 关仝 秋山晚翠图绢本.jpg
处理完：元 赵孟頫 鹊华秋色全卷(二版）纸本28.4x289副本 5.49.12 PM.jpg
处理完：元 赵孟頮 双松平远图26.8x107.5cm.jpg
处理完：元  盛懋 坐看云起图页69x76.jpg
处理完：元 赵孟頮 水村图卷(全卷二版)纸本24.9cmX120.jpg
全部处理完成。
